### Compilations

In [ ]:
!apt update && apt install -y libsm6 libxext6
!apt-get install -y libxrender-dev

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/cython_utils
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

In [ ]:
%cd /workdir/msc/RotationDetection/libs/utils/
!rm *.so
!rm *.c
!rm *.cpp
!python setup.py build_ext --inplace

### Dataset

In [ ]:
# change train and test paths inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert_ann2dota.py

In [3]:
# change root path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python convert2jpg.py '/workdir/msc/datasets/Fluo-N2DH-SIM+/02/images' '.tif'

/workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
100%|█████████████████████████████████████████| 150/150 [00:15<00:00,  9.60it/s]
/workdir/msc/datasets/Fluo-N2DH-SIM+/02/images
MEAN:  23.039735626666666
STD:  12.663884742722097


In [4]:
# change train path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python data_crop.py '/workdir/msc/datasets/Fluo-N2DH-SIM+/01'

/workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
class_list 1
find image 65
find label 65
100%|███████████████████████████████████████████| 65/65 [01:34<00:00,  1.45s/it]


In [6]:
# change test path inside the script
%cd /workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
!python txt2xml.py \
'/workdir/msc/datasets/Fluo-N2DH-SIM+/01/annotations/dota_format' \
'/workdir/msc/datasets/Fluo-N2DH-SIM+/01/annotations/xml_rotdet' \
'/workdir/msc/datasets/Fluo-N2DH-SIM+/01/images'

/workdir/msc/RotationDetection/dataloader/dataset/UFRGS_CELL
100%|███████████████████████████████████████████| 65/65 [00:00<00:00, 76.41it/s]


In [7]:
%cd /workdir/msc/RotationDetection/dataloader/dataset/
!python convert_data_to_tfrecord.py --VOC_dir='/workdir/msc/datasets/Fluo-N2DH-SIM+/01/crop' \
                                    --xml_dir='labeltxt' \
                                    --image_dir='images' \
                                    --save_name='train'  \
                                    --img_format='.png'  \
                                    --save_dir='/workdir/msc/datasets' \
                                    --dataset='UFRGS_CELL_1class'

/workdir/msc/RotationDetection/dataloader/dataset
/workdir/msc/RotationDetection

100%|███████████████████████████████████████| 2340/2340 [00:24<00:00, 96.58it/s]

Conversion is complete!


## Train

In [9]:
#!rm -r /workdir/msc/RotationDetection/output/**/*Fluo-N2DH-SIM+*

rm: cannot remove '/workdir/msc/RotationDetection/output/**/*Fluo-N2DH-SIM+*': No such file or directory


In [ ]:
# change the path to the detector!
%cd /workdir/msc/RotationDetection/tools/r2cnn
!python train.py

************************************************************************
2022-07-15 17:56:54: global_step:20  current_step:20
speed: 3.201s, remaining training time: 01:01:56:45
total_losses:0.329
rpn_cls_loss:0.207
rpn_reg_loss:0.009
fast_cls_loss:0.090
fast_reg_loss:0.023

************************************************************************
2022-07-15 17:57:03: global_step:40  current_step:40
speed: 0.319s, remaining training time: 00:02:34:53
total_losses:0.897
rpn_cls_loss:0.115
rpn_reg_loss:0.014
fast_cls_loss:0.352
fast_reg_loss:0.416

************************************************************************
2022-07-15 17:57:08: global_step:60  current_step:60
speed: 0.254s, remaining training time: 00:02:03:16
total_losses:0.492
rpn_cls_loss:0.023
rpn_reg_loss:0.004
fast_cls_loss:0.284
fast_reg_loss:0.182

************************************************************************
2022-07-15 17:57:13: global_step:80  current_step:80
speed: 0.254s, remaining training time: 00:02:

## Test

| Model      | AP50 normal | AP50 mitoses | P50 normal | R50 normal | P50 mitoses | R50 mitoses |   AP50    |   AP75    |   AP50:95   |
|----------- | ----------- | ------------ | ---------- | ---------- | ----------- | ----------- |   -----   |   -----   |   -------   |
| RetinaNet  |   75.25     |   56.47      | 37.64      | 85.00      | 10.99       | 82.85       |   65.86   |   19.17   |   29.56     |
| R3Det      |   74.97     |   63.21      | 32.68      | 83.42      | 14.35       | 84.76       |   69.09   |   17.91   |   30.89     |
| R3Det DCL  |   75.50     |   62.63      | 31.58      | 83.54      | 15.75       | 84.76       |   69.06   | **20.57** |   30.73     |
| DCL        |   71.81     |   63.35      | 39.58      | 81.62      | 16.50       | 80.95       |   67.58   |   17.74   |   28.48     |
| CSL        |   72.42     |   53.98      | 35.20      | 84.10      | 10.94       | 84.76       |   63.20   |   18.59   |   27.00     |
| R2CNN      | **77.33**   | **69.65**    | **67.49**  | 82.41      | **52.79**   | 80.95       | **73.49** |   17.30   |   29.71     |
| Refine RetinaNet | 75.15 |   66.75      | 27.93      | **85.68**  | 11.94       | **89.52**   |   70.95   |   18.86   | **31.00**   |
| RSDet      |   74.33     |   63.60      | 38.65      | 83.31      | 9.63        | 87.62       |   68.97   |   16.15   |   29.86     |

In [ ]:
%cd /workdir/msc/RotationDetection/tools/r2cnn
!python test_ufrgscell.py --img_dir='/workdir/msc/datasets/Fluo-N2DH-SIM+/02/images'  \
                          --gpu=0 \
                          --image_ext='.jpg' \
                          --test_annotation_path='/workdir/msc/datasets/Fluo-N2DH-SIM+/02/annotations/xml_rotdet' \
                          none

## Inference

In [ ]:
import os
import cv2
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

path_labels = '/workdir/datasets/msc/UFRGS_CELL_2classes/test/annotations/dota_format'
path_draws = '/workdir/msc/RotationDetection/tools/gwd/test_ufrgscell/RetinaNet_UFRGS_CELL_smooth_l1_loss/ufrgscell_img_vis'

path_labels = sorted(glob(os.path.join(path_labels, '*.txt')))
path_draws  = sorted(glob(os.path.join(path_draws, '*.jpg')))

for i in range(len(path_labels)):
    path_ann = path_labels[i]
    path_img = path_draws[i]

    img = cv2.imread(path_img)[:,:,::-1]
    h,w,c = img.shape

    labels = []
    for lb in open(path_ann, 'r').read().split('\n')[2:-1]:
        lb = np.reshape(list(map(float, lb.split(' ')[:-2])), (-1,2))
        labels.append(lb)
    labels = np.int0(labels)
    
    draw = np.copy(img)
    cv2.drawContours(draw, labels, -1, (255,0,0), 2)

    plt.figure(figsize=(15,15))
    plt.imshow(draw)